In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
from monty.serialization import loadfn, dumpfn

In [3]:
from ase.filters import FrechetCellFilter
from ase.optimize import LBFGS
from pymatgen.io.ase import AseAtomsAdaptor
# from orb_models.forcefield import pretrained
# from orb_models.forcefield.calculator import ORBCalculator
from fairchem.core import pretrained_mlip, FAIRChemCalculator
# from eqnorm.calculator import EqnormCalculator
# from mace.calculators import mace_mp
# from mattersim.forcefield import MatterSimCalculator
# from sevenn.calculator import SevenNetCalculator


/home/atshaam/miniconda3/envs/proj_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
MODEL = "uma" # "orb" or "uma" or "mace" or "seven" or "mattersim" or "eqnorm"

In [5]:
supercell_size = 10.1

In [6]:
# orb_ff = pretrained.orb_v3_conservative_inf_omat(device="cuda",precision="float32-high")  # or "float32-highest" / "float64
# orb_calc = ORBCalculator(orb_ff, device="cuda")

uma_ff = pretrained_mlip.get_predict_unit("uma-m-1p1", device="cuda")
uma_calc = FAIRChemCalculator(uma_ff, task_name="omat")

# eqnorm_calc = EqnormCalculator(model_name='eqnorm', model_variant='eqnorm-mptrj', device='cuda')


# mace_calc = mace_mp(model="MACE-matpes-r2scan-omat-ft.model", dispersion=False, device='cuda')
# seven_calc = SevenNetCalculator('7net-mf-ompa', modal='omat24')

# mattersim_calc = MatterSimCalculator(device="cuda")

MODELS = {
    # "orb":orb_calc,
    "uma": uma_calc,
    # # "seven":seven_calc,
    # "mattersim":mattersim_calc,
    # "eqnorm":eqnorm_calc
    # "mace": mace_calc
}

In [7]:
def relax_structure(structure): # takes in pymatgen structure
    atoms = structure.to_ase_atoms()
    atoms.calc = MODELS[MODEL]
    
    opt = LBFGS(FrechetCellFilter(atoms))# Delete this FrechetCellFilter when doing the defect structures, need FrechetCellFilter when doing competing phases
    opt.run(0.01, 1000)
    
    relaxed_structure = AseAtomsAdaptor.get_structure(atoms)
    energy = atoms.get_potential_energy()

    return relaxed_structure, energy

In [8]:
from doped.chemical_potentials import CompetingPhases
from pymatgen.entries.computed_entries import ComputedEntry
from doped.chemical_potentials import CompetingPhasesAnalyzer
from tqdm import tqdm
from pymatgen.entries.compatibility import MaterialsProject2020Compatibility

In [9]:
from pymatgen.core import Element

MP_U_VALUES = {
    "Co": 3.32,
    "Cr": 3.7,
    "Fe": 5.3,
    "Mn": 3.98,
    "Mo": 4.38,
    "Ni": 6.2,
    "V": 3.25,
    "W": 6.2,
}

from pymatgen.core import Species

def add_parameters(entry):
    parameters = {}
    if any([Element(el) in entry.composition.elements for el in MP_U_VALUES]):
        parameters["run_type"] = "GGA+U"
        parameters["hubbards"] = {el.name: MP_U_VALUES.get(el.symbol, 0.0) for el in entry.composition.elements}
        parameters["is_hubbard"] = True
    else:
        parameters["run_type"] = "GGA"
        parameters["hubbards"] = None
        parameters["is_hubbard"] = False
    entry.parameters = parameters
    return entry

In [10]:
calculated_entries = {}

def calc_competing_phase(comp): # Pass in formula of species
    cp = CompetingPhases(composition=comp, extrinsic='Cr', energy_above_hull=0.05)
    
    entries = []
    for entry in tqdm(cp.entries):
        key =  entry.data['material_id'] + entry.structure.composition.reduced_formula
        
        if key in calculated_entries:
            new_entry = calculated_entries[key]
        else:
            try:
                relaxed_structure, energy = relax_structure(entry.structure)
            except ValueError as e:
                print(f"Skipping structure due to error: {e}")
                continue
            new_entry = ComputedEntry(
                relaxed_structure.composition,
                energy, 
                data = {'material_id':entry.data['material_id']}
            )
            new_entry.structure = relaxed_structure   # IMPORTANT!
            new_entry = add_parameters(new_entry)     # IMPORTANT!
            calculated_entries[key] = new_entry 
        entries.append(new_entry)
    compat = MaterialsProject2020Compatibility(check_potcar=False, check_potcar_hash=False)
    new_entries = compat.process_entries(entries, inplace=False, on_error="warn")
    return cp, new_entries 

In [11]:
def calc_competing_phase_analysis(comp, entries):
    
    cpa = CompetingPhasesAnalyzer(composition=comp, entries=entries)
    
    return cpa.chempots, cpa.unstable_host

In [12]:
relaxed_defect = loadfn(f'../../data/{MODEL}/{MODEL}_corrected_doped_defect_energies_{supercell_size}A.json')

In [ ]:
for key in relaxed_defect.keys():
    formula = relaxed_defect[key]['formula']
    cp, entries = calc_competing_phase(formula) # do you have to return cp?
    chem_pot, unstable = calc_competing_phase_analysis(formula, entries)
    relaxed_defect[key]['competing phases'] = {}
    relaxed_defect[key]['competing phases']['chemical potentials'] = chem_pot
    relaxed_defect[key]['competing phases']['entries'] = entries
    relaxed_defect[key]['competing phases']['unstable'] = unstable

In [15]:
dumpfn(relaxed_defect, f'../../data/{MODEL}/{MODEL}_corrected_doped_defect_energies_{supercell_size}A.json')